In [1]:
#import os, uuid
from pathlib import Path
from azureml.core import Dataset, Datastore
from azureml.core import Workspace
import azureml.core
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Connect to the Workspace

In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print(f'Ready to use Azure ML {azureml.core.VERSION} to work with {ws.name}')

Ready to use Azure ML 1.18.0 to work with jbWorkspace


# Upload and Register a Dataset

In [4]:
blob_datastore = Datastore.get(ws, 'workspaceblobstore')

In [ ]:
blob_datastore.upload('/home/jeremy/Documents/data/quiddler', target_path='quiddler/', overwrite=True)

In [143]:
file_dataset = Dataset.File.from_files(path=[(blob_datastore,'quiddler/')], validate=True)

In [ ]:
file_dataset.take(1).download()

In [ ]:
file_dataset.register(ws,'quiddler_dataset')

# Training

In [3]:
ds = ws.datasets['quiddler_dataset']

In [4]:
type(ds)

azureml.data.file_dataset.FileDataset

In [5]:
experiment_name = 'quiddler-2'
exp = Experiment(workspace=ws, name=experiment_name)

In [6]:
#compute_target = ws.compute_targets['jb-ml-gpu3']
compute_target = 'local'

In [7]:
from azureml.train.estimator import Estimator

est = Estimator(source_directory='source', 
                entry_script='training.py',
                inputs=[ds.as_named_input('quiddler_ds').as_download()],
                script_params = {'--epochs': 10, 
                                 '--batch_size': 6,
                                 '--image_size': 512},
                pip_packages=['icevision[all]==0.1.4'],
                compute_target=compute_target,
                use_gpu=True)

In [8]:
# Run the experiment
run = exp.submit(config=est)

# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

Note: The GPU base image must be used on Microsoft Azure Services only. See LICENSE.txt file.
Note: The GPU base image must be used on Microsoft Azure Services only. See LICENSE.txt file.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'quiddler-2_1606217317_af5f1378',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-11-24T11:28:40.658877Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '66710249-f85b-4e17-8e00-459d8f5e9edc'},
 'inputDatasets': [{'dataset': {'id': 'efdf9361-557d-4bf4-af07-53f02f0a01b7'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'quiddler_ds', 'mechanism': 'Download'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'training.py',
  'useAbsolutePath': False,
  'arguments': ['--epochs', '10', '--batch_size', '6', '--image_size', '512'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'quiddler_ds': {'dataLocation': {'dataset': {'id': 'efdf9361-557d-4bf4-af07-53f02f0a01b7',
      'name': 'quiddler_dataset',
      'version': '1'},
     'dataPath': None},
    'mechanism': 'Download',
    'environmentVariableName': 'quiddler_ds',
  

In [9]:
run.get_metrics()

{'epochs': 10,
 'batch_size': 6,
 'image_size': 512,
 'train_loss': [1.4829208850860596,
  1.068541407585144,
  1.0864710807800293,
  1.0839167833328247,
  1.0106157064437866,
  0.95765221118927,
  0.888633668422699,
  0.8286517858505249,
  0.766325056552887,
  0.7086866497993469,
  0.6655247807502747,
  0.5910255908966064,
  0.5902135372161865,
  0.5671930909156799,
  0.5661458969116211,
  0.558785617351532,
  0.5411462783813477,
  0.5106704235076904,
  0.4837430417537689,
  0.46543973684310913,
  0.4424753189086914,
  0.4102891683578491],
 'valid_loss': [1.1236693859100342,
  1.2535189390182495,
  1.2105265855789185,
  0.8515728116035461,
  1.1556785106658936,
  0.8750515580177307,
  0.836686372756958,
  0.7628051042556763,
  0.6337860822677612,
  0.6075886487960815,
  0.6037845611572266,
  0.7040266394615173,
  0.6900558471679688,
  0.4999760389328003,
  0.6549802422523499,
  0.6650242805480957,
  0.48089292645454407,
  0.44275468587875366,
  0.46885430812835693,
  0.415402472019195

# Register the best model

In [34]:
import pandas as pd

In [45]:
runs = pd.DataFrame(data = [(r, r.id, r.get_metrics(name='clf_error').get('clf_error')) for r in exp.get_runs()],
                    columns = ['run', 'id','clf_error'])

In [46]:
runs.dropna(inplace=True)

In [47]:
runs.sort_values('clf_error')

,run,id,clf_error
1,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603764126_91660cbb,0.000000
4,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603678429_dd5dae12,0.000000
0,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1604159521_666dca58,0.090909
5,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603659582_9a5067fb,0.290909
8,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603651796_8854241a,0.372727
6,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603657302_456adf85,0.572727
15,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603638787_4106ff9a,0.672727
7,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603654434_893d3810,1.090909
16,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603597870_da99d143,1.163636
2,"Run(Experiment: quiddler-2,\nId: quiddler-2_16...",quiddler-2_1603682405_c08997ce,2.236364


In [50]:
best_run = runs.run[0]

In [51]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
quiddler-2,quiddler-2_1604159521_666dca58,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [52]:
type(best_run)

azureml.core.script_run.ScriptRun

In [58]:
last_COCOMetric_mAP = best_run.get_metrics()['COCOMetric_mAP'][-1]

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/quiddler.pt', model_name='quiddler_model',
                        tags={'Training context':'IceVision on Compute Cluster'},
                        properties={'last_COCOMetric_mAP': last_COCOMetric_mAP})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')